In [1]:
import requests

import pandas as pd

from bs4 import BeautifulSoup

import time

pd.set_option('display.max_columns', 50)

In [2]:
years = list(range(2024,2021, -1))
years

[2024, 2023, 2022]

In [3]:
connect_timeout = 6
read_timeout = 100

def stats(link, year):
     
    data = requests.get(link, timeout=(connect_timeout, read_timeout))
    soup = BeautifulSoup(data.text)
    
    stats_table = pd.read_html(data.text, match="Regular season Table")[0]
    
    standard_stats = pd.read_html(data.text, match = "Squad Standard Stats")[0]
    standard_stats.columns = standard_stats.columns.droplevel()
    
    goalkeeping_stats = pd.read_html(data.text, match="Squad Goalkeeping")[0]
    goalkeeping_stats.columns = goalkeeping_stats.columns.droplevel()
    
    shooting_stats = pd.read_html(data.text, match="Squad Shooting")[0]
    shooting_stats.columns = shooting_stats.columns.droplevel()
    
    stats_standard = stats_table.merge(standard_stats[["Squad", "Poss", "Gls", "Ast"]], on="Squad", how="left")
    stats_standard_gk = stats_standard.merge(goalkeeping_stats[["Squad", "CS%"]], on="Squad", how="left")
    all_stats = stats_standard_gk.merge(shooting_stats[["Squad", "SoT/90", "G/SoT", "SoT%"]], on="Squad", how="left")
        
    all_stats["Season"] = year
    
    return all_stats
    

In [4]:
overall_stats = []

In [5]:
def concate():
    
    all_stats_2024 = stats("https://fbref.com/en/comps/9/Premier-League-Stats", 2024)
    all_stats_2024 = all_stats_2024.drop(["Last 5"], axis=1)
    all_stats_2023 = stats("https://fbref.com/en/comps/9/2022-2023/2022-2023-Premier-League-Stats", 2023)
    all_stats_2022 = stats("https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats", 2022)
    all_stats_2021 = stats("https://fbref.com/en/comps/9/2020-2021/2020-2021-Premier-League-Stats", 2021)
    all_stats_2020 = stats("https://fbref.com/en/comps/9/2019-2020/2019-2020-Premier-League-Stats", 2020)
    all_stats_2019 = stats("https://fbref.com/en/comps/9/2018-2019/2018-2019-Premier-League-Stats", 2019)
    
    overall_stats.append(all_stats_2024)
    overall_stats.append(all_stats_2023)
    overall_stats.append(all_stats_2022)
    overall_stats.append(all_stats_2021)
    overall_stats.append(all_stats_2020)
    overall_stats.append(all_stats_2019)
    
    overall_stats_df = pd.concat(overall_stats)
    return overall_stats_df
    

In [6]:
overall_stats_df = concate()

In [7]:
overall_stats_df.head()

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Attendance,Top Team Scorer,Goalkeeper,Notes,Poss,Gls,Gls,Ast,Ast,CS%,SoT/90,G/SoT,SoT%,Season
0,1,Liverpool,24,16,6,2,55,23,32,54,2.25,51.7,29.4,22.4,0.93,53634,Mohamed Salah - 14,Alisson,NaN,60.0,51,2.12,40,1.67,33.3,6.42,0.31,34.2,2024
1,2,Manchester City,23,16,4,3,56,25,31,52,2.26,45.1,21.9,23.2,1.01,53091,Erling Haaland - 16,Ederson,NaN,65.2,55,2.39,42,1.83,26.1,6.61,0.34,38.9,2024
2,3,Arsenal,24,16,4,4,53,22,31,52,2.17,47.9,17.7,30.1,1.26,60196,Bukayo Saka - 10,David Raya,NaN,60.5,52,2.17,36,1.50,37.5,5.25,0.36,31.7,2024
3,4,Tottenham,24,14,5,5,51,36,15,47,1.96,41.9,40.9,1.0,0.04,61574,Son Heung-min - 12,Guglielmo Vicario,NaN,59.3,48,2.00,40,1.67,20.8,5.79,0.34,36.7,2024
4,5,Aston Villa,24,14,4,6,50,32,18,46,1.92,43.2,32.5,10.7,0.45,41611,Ollie Watkins - 11,Emiliano Martínez,NaN,55.8,47,1.96,34,1.42,25.0,5.37,0.33,36.4,2024


In [8]:
overall_stats_df.shape

(120, 29)

In [9]:
overall_stats_df.to_csv("../datasets/overall_stats2.csv", index=False)